# 3. Preprocessing

In [1]:
#Import the necessary packages
import pandas as pd
import numpy as np
import nltk
import spacy
import contractions as cont
import gensim.downloader as api
import re
import unicodedata
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords

In [2]:
#Load the data
df = pd.read_csv('Corona_NLP.csv', index_col=0)
df.head()

,OriginalTweet,Sentiment
0,TRENDING: New Yorkers encounter empty supermar...,Extremely Negative
1,When I couldn't find hand sanitizer at Fred Me...,Positive
2,Find out how you can protect yourself and love...,Extremely Positive
3,#Panic buying hits #NewYork City as anxious sh...,Negative
4,#toiletpaper #dunnypaper #coronavirus #coronav...,Neutral


In [3]:
df.dropna(inplace=True) #Something wonky happened while saving the dataframe to csv and two of the Sentiments were
#saved as separate rows, creating a few NaNs. 

In its current form, the OriginalTweet column contains unstructured data in the form of Tweets. Most NLP classification algorithms, however, operate on word or document vectors and so it is necessary to vectorize each Tweet before preceding to the modelling stage. To aid in the process of vectorization, I defined three functions and applied them to the OriginalTweet column: preprocess, lemmatize_and_remove_ents, fix_spelling, and vectorize. These functions either remove any words that do not contribute to sentiment (e.g., hashtags, handles, links, proper names) or reduce noise by removing stopwords, proper names, and mispellings. 

The first function preforms several important tasks: it converts each Tweet to lower case; expands contractions; and removes a variety of non-essential information (hashtags, twitter handles, links, accented characters, non-alphabetic characters apart from the dash "-", and stopwords). The second step proved necessary to preserve the word "not," which is often essential for distinguishing positive sentiments from negative ones. Consider, for example, the difference in meaning between 'happy' and 'not happy'. 

In [4]:
stopword_list = nltk.corpus.stopwords.words('english')
stopword_list.remove('not') 

def preprocess(text):
    text = text.lower() #lowercase
    text = " ".join([cont.fix(word) for word in text.split()]) #expand contractions
    text = re.sub(r"(#\S+)", '', text) #remove hashtags
    text = re.sub(r"(@\S+)", '', text) #remove handles
    text = re.sub(r"(http\S+)", '', text) #remove links
    text = unicodedata.normalize('NFKD', text) #remove diacritics
    text = text.encode('ascii', errors='ignore').decode('utf-8', errors='ignore') 
    tokens = [word.strip() for word in text.split() if word not in stopword_list] #remove stopwords
    text = " ".join(tokens)
    text = re.sub(r"[^a-z- ]+", '', text) #remove special characters 
    return text

In [5]:
df['ProcessedTweet'] = df['OriginalTweet'].apply(preprocess)

df.head() 

,OriginalTweet,Sentiment,ProcessedTweet
0,TRENDING: New Yorkers encounter empty supermar...,Extremely Negative,trending new yorkers encounter empty supermark...
1,When I couldn't find hand sanitizer at Fred Me...,Positive,could not find hand sanitizer fred meyer turne...
2,Find out how you can protect yourself and love...,Extremely Positive,find protect loved ones
3,#Panic buying hits #NewYork City as anxious sh...,Negative,buying hits city anxious shoppers stock foodam...
4,#toiletpaper #dunnypaper #coronavirus #coronav...,Neutral,one week everyone buying baby milk powder next...


In [6]:
nlp = spacy.load("en_core_web_md", exclude=['Parser'])

def lemmatize_and_remove_ents(text):
    text = nlp(text)
    text = " ".join([word.lemma_ if word.lemma_ != "-PRON-" else word.text for word in text if not word.ent_type_ ])
    return text

In [7]:
df['ProcessedTweet'] = df['ProcessedTweet'].apply(lemmatize_and_remove_ents)

df.head() 

,OriginalTweet,Sentiment,ProcessedTweet
0,TRENDING: New Yorkers encounter empty supermar...,Extremely Negative,trend new yorkers encounter empty supermarket ...
1,When I couldn't find hand sanitizer at Fred Me...,Positive,could not find hand sanitizer fred meyer turn ...
2,Find out how you can protect yourself and love...,Extremely Positive,find protect love one
3,#Panic buying hits #NewYork City as anxious sh...,Negative,buy hit city anxious shopper stock foodampmedi...
4,#toiletpaper #dunnypaper #coronavirus #coronav...,Neutral,everyone buy baby milk powder next everyone bu...


The third and final function converts the processed tweets into document vectors using a combination of a pretrained GloVe embedding and Tfidf weighting. It does so by calculating the sum of individual word vectors multiplied by its Tfidf weight for each word within a Tweet. Weighting the individual word vectors helps ensure that the embeddings for long and short Tweets do not differ too drastically. 

Originally, I had planned use a larger embedding, such as the 'word2vec-google-news-300', but my computer couldn't handle it. 

In [8]:
glove = api.load('glove-twitter-25')

tv = TfidfVectorizer(stop_words=stopword_list)
tv_transformed = tv.fit_transform(df['ProcessedTweet'])
tfidf_values = dict(zip(tv.get_feature_names(), tv.idf_))

In [9]:
def vectorizer(text):
    vector = sum([glove[word]*tfidf_values[word] for word in text.split() if word in tfidf_values.keys() and word in glove.key_to_index])
    return vector

In [10]:
df['DocVector'] = df['ProcessedTweet'].apply(vectorizer)

df.head()

,OriginalTweet,Sentiment,ProcessedTweet,DocVector
0,TRENDING: New Yorkers encounter empty supermar...,Extremely Negative,trend new yorkers encounter empty supermarket ...,"[-25.564545, 12.41317, 0.9165025, -23.523197, ..."
1,When I couldn't find hand sanitizer at Fred Me...,Positive,could not find hand sanitizer fred meyer turn ...,"[-18.407335, 11.812518, -2.667809, 6.388415, -..."
2,Find out how you can protect yourself and love...,Extremely Positive,find protect love one,"[-2.7793016, 3.3964226, -5.5020514, 8.437631, ..."
3,#Panic buying hits #NewYork City as anxious sh...,Negative,buy hit city anxious shopper stock foodampmedi...,"[-40.624233, 35.157276, -24.591461, -49.36236,..."
4,#toiletpaper #dunnypaper #coronavirus #coronav...,Neutral,everyone buy baby milk powder next everyone bu...,"[-22.856718, 14.615568, 17.575512, 6.351932, -..."


Next I need to transform DocVector from a column of lists into a separate dataframe consisting of 25 columns, one for each of the 25 vectorized features. This dataframe will contain all of the explanatory variables for the modelling step of the project. 

In [11]:
X = df.DocVector.apply(pd.Series)
X.columns = ['Feature ' + str(i) for i in range(25)]

X.head()

,Feature 0,Feature 1,Feature 2,Feature 3,Feature 4,Feature 5,Feature 6,Feature 7,Feature 8,Feature 9,...,Feature 15,Feature 16,Feature 17,Feature 18,Feature 19,Feature 20,Feature 21,Feature 22,Feature 23,Feature 24
0,-25.564545,12.413170,0.916502,-23.523197,23.942314,-19.245277,39.597092,-67.066742,27.730808,-9.485415,...,12.364698,41.769032,-26.896332,23.701748,-1.653188,-19.830172,-27.282768,-11.635708,-29.070320,-34.822224
1,-18.407335,11.812518,-2.667809,6.388415,-18.285372,-4.658720,22.325075,-37.602386,21.859562,-21.498049,...,-1.258633,8.867926,-8.832096,15.507577,-15.484630,-4.390882,11.780195,12.107148,5.713201,-26.333225
2,-2.779302,3.396423,-5.502051,8.437631,-8.695304,-5.224719,29.446379,2.569920,-7.636728,0.954528,...,4.210380,5.014014,-11.005681,5.610911,-7.698394,-2.586307,-0.576156,-5.779848,0.555017,-8.106094
3,-40.624233,35.157276,-24.591461,-49.362358,8.023435,7.366742,49.222126,-25.602089,34.722832,-19.868265,...,6.108077,28.576767,-23.534607,-16.189983,-31.512745,-27.594158,-26.385250,-12.707217,-22.948502,-21.563004
4,-22.856718,14.615568,17.575512,6.351932,-3.586997,6.752620,55.943478,-31.262821,4.476537,18.464659,...,-7.847046,31.560469,-41.715862,2.640697,8.316002,-27.561853,-7.074276,15.240105,24.190716,-8.238882


Next I need to replace the sentiment labels with numerical dummy values. 

In [12]:
replacements = {'Extremely Negative': 0, 'Negative': 1, 'Neutral' : 2, 'Positive' : 3, 'Extremely Positive' : 4}
df['Sentiment'].replace(replacements, inplace=True)

df.head()

,OriginalTweet,Sentiment,ProcessedTweet,DocVector
0,TRENDING: New Yorkers encounter empty supermar...,0,trend new yorkers encounter empty supermarket ...,"[-25.564545, 12.41317, 0.9165025, -23.523197, ..."
1,When I couldn't find hand sanitizer at Fred Me...,3,could not find hand sanitizer fred meyer turn ...,"[-18.407335, 11.812518, -2.667809, 6.388415, -..."
2,Find out how you can protect yourself and love...,4,find protect love one,"[-2.7793016, 3.3964226, -5.5020514, 8.437631, ..."
3,#Panic buying hits #NewYork City as anxious sh...,1,buy hit city anxious shopper stock foodampmedi...,"[-40.624233, 35.157276, -24.591461, -49.36236,..."
4,#toiletpaper #dunnypaper #coronavirus #coronav...,2,everyone buy baby milk powder next everyone bu...,"[-22.856718, 14.615568, 17.575512, 6.351932, -..."


The final step in preprocessing is to divide the data into test and training sets. 

In [13]:
y = df['Sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=42)

In [14]:
df.to_csv('processed_Corona_NLP.csv')
X.to_csv('DocVectors.csv')